In [1]:
import math
from gurobipy import *

In [2]:
class Data:
    customerNum = 0
    vehicleNum  = 0
    capacity    = 0
    cor_X       = []
    cor_Y       = []
    demand      = []
    serviceTime = []
    readyTime   = []
    dueTime     = []
    disMatrix   = [[]]


In [5]:
def readData(data, path, customerNum):
    data.customerNum = customerNum
    f = open(path, 'r')
    lines = f.readlines()
    count = 0

    line = lines[4];
    line = line[:-1].strip()
    str = re.split(r"\s{1,}", line)

    data.vehicleNum = int(str[0])
    data.capacity = float(str[1])

    for i in range(9, len(lines)):
        line = lines[i];
        count = count + 1

        line = line[:-1].strip()
        str = re.split(r"\s{1,}", line)
        data.cor_X.append(float(str[1]))
        data.cor_Y.append(float(str[2]))
        data.demand.append(float(str[3]))
        data.readyTime.append(float(str[4]))
        data.dueTime.append(float(str[5]))
        data.serviceTime.append(float(str[6]))

    data.cor_X.append(data.cor_X[0])
    data.cor_Y.append(data.cor_Y[0])
    data.demand.append(data.demand[0])
    data.readyTime.append(data.readyTime[0])
    data.dueTime.append(data.dueTime[0])
    data.serviceTime.append(data.serviceTime[0])

    data.disMatrix = [([0] * (data.customerNum + 2)) for p in range(data.customerNum + 2)]

    for i in range(0, data.customerNum + 2):
        for j in range(0, data.customerNum + 2):
            if (i != j):
                temp = (data.cor_X[i] - data.cor_X[j]) ** 2 + (data.cor_Y[i] - data.cor_Y[j]) ** 2
                data.disMatrix[i][j] = math.sqrt(temp)
                temp = 0
            else:
                data.disMatrix[i][j] = math.sqrt(100000.0)
    return data

#---读取数据
data = Data()
path = './solomon-100/in/c101.txt' #读取算例数据集
customerNum = 100  #设置客户数量
readData(data, path, customerNum)
BigM = 100000

In [6]:
x = {}  # 存放决策变量x_ijk
s = {}  # s_ik表示车辆k开始服务客户i的时间

model = Model()

In [7]:
# 定义决策变量，并加入模型当中：
for i in range(0, data.customerNum + 2):
    for k in range(0, data.vehicleNum):
        name = 's_' + str(i) + '_' + str(k)
        s[i, k] = model.addVar(0, 1500, vtype=GRB.CONTINUOUS, name=name)

        for j in range(0, data.customerNum + 2):
            if (i != j):
                name = 'x_' + str(i) + '_' + str(j) + '_' + str(k)
                x[i, j, k] = model.addVar(0, 1, vtype=GRB.BINARY, name=name)

In [8]:
# 根据式(1)定义目标函数，并加入模型当中：
obj = LinExpr(0)
for i in range(data.customerNum + 2):
    for k in range(data.vehicleNum):
        for j in range(data.customerNum + 2):
            if (i != j):
                obj.addTerms(data.disMatrix[i][j], x[i, j, k])

model.setObjective(obj, GRB.MINIMIZE)

In [9]:
# 约束二：
for i in range(1, data.customerNum + 2 - 1):
    lhs = LinExpr(0)
    for k in range(data.vehicleNum):
        for j in range(1, data.customerNum + 2):
            if (i != j):
                lhs.addTerms(1, x[i, j, k])
    model.addConstr(lhs == 1, name='visit_' + str(i))


In [10]:
# 约束三：
for k in range(0, data.vehicleNum):
    lhs = LinExpr(0)
    for j in range(1, data.customerNum + 2):
        lhs.addTerms(1, x[0, j, k])
    model.addConstr(lhs == 1, name='vehicle_' + str(k))


In [11]:
# 约束四：
for k in range(data.vehicleNum):
    for h in range(1, data.customerNum + 2 - 1):
        expr1 = LinExpr(0)
        expr2 = LinExpr(0)
        for i in range(data.customerNum + 2 - 1):
            if (h != i):
                expr1.addTerms(1, x[i, h, k])

        for j in range(1, data.customerNum + 2):
            if (h != j):
                expr2.addTerms(1, x[h, j, k])

        model.addConstr(expr1 == expr2, name='flow_conservation_' + str(i))
        expr1.clear()
        expr2.clear()


In [12]:
# 约束五：
for k in range(data.vehicleNum):
    lhs = LinExpr(0)
    for j in range(data.customerNum + 2 - 1):
        lhs.addTerms(1, x[j, data.customerNum + 2 - 1, k])
    model.addConstr(lhs == 1, name='enter_' + str(k))


In [13]:
# 约束六
for k in range(data.vehicleNum):
    for i in range(data.customerNum + 2):
        for j in range(data.customerNum + 2):
            if (i != j):
                model.addConstr(
                    s[i, k] + data.disMatrix[i][j] + data.serviceTime[i] - s[j, k] - 2 * BigM + 2 * BigM * x[
                        i, j, k] <= 0, name='time_windows_')


In [14]:
# 约束七：
for k in range(data.vehicleNum):
    for i in range(1, data.customerNum + 2 - 1):
        lhs1 = LinExpr()
        lhs2 = LinExpr()
        for j in range(1, data.customerNum + 2):
            if (i != j):
                model.addConstr(data.readyTime[i] * x[i, j, k] <= s[i, k], name='ready_time_' + str(i))
                model.addConstr(data.dueTime[i] + (BigM - BigM * x[i, j, k]) >= s[i, k], name='due_time_' + str(i))


In [15]:
# 约束八：
for k in range(data.vehicleNum):
    model.addConstr(data.readyTime[0] <= s[0, k], name='ready_time_' + str(0))
    model.addConstr(data.dueTime[0] >= s[0, k], name='due_time_' + str(0))
    model.addConstr(data.readyTime[data.customerNum + 1] <= s[data.customerNum + 1, k],
                    name='ready_time_' + str(data.customerNum + 1))
    model.addConstr(data.dueTime[data.customerNum + 1] >= s[data.customerNum + 1, k],
                    name='due_time_' + str(data.customerNum + 1))


In [16]:
# 约束九
for k in range(data.vehicleNum):
    lhs = LinExpr(0)
    for i in range(1, data.customerNum + 2 - 1):
        for j in range(1, data.customerNum + 2):
            if (i != j):
                lhs.addTerms(data.demand[i], x[i, j, k])
    model.addConstr(lhs <= data.capacity, name='capacity_' + str(k))


In [17]:
model.optimize()
print("\n\n-----optimal value-----")
print(model.ObjVal)

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 760325 rows, 260100 columns and 2777800 nonzeros
Model fingerprint: 0x7d33828f
Variable types: 2550 continuous, 257550 integer (257550 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+05]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 2e+03]
  RHS range        [1e+00, 2e+05]
Presolve removed 502650 rows and 5075 columns (presolve time = 5s) ...
Presolve removed 502650 rows and 5075 columns (presolve time = 181s) ...
Presolve removed 573673 rows and 76098 columns
Presolve time: 183.24s
Presolved: 186652 rows, 184002 columns, 1618343 nonzeros
Variable types: 2500 continuous, 181502 integer (181502 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.7709624e+03  

In [18]:
print("routes = ")
for k in range(data.vehicleNum):
    sys.stdout.write("[")
    i = 0;
    while i <= customerNum + 2 - 1:
        if(i == customerNum + 1):
            sys.stdout.write(str(i))
            break
        for j in range(1,data.customerNum + 2):
            if(i != j and x[i,j,k].x > 0):
                sys.stdout.write(str(i) + ",")
                break;
        i = j
    sys.stdout.write("]")
    print("")

routes = 
[0,101]
[0,101]
[0,101]
[0,101]
[0,101]
[0,5,3,7,8,10,11,9,6,4,2,1,75,101]
[0,57,55,54,53,56,58,60,59,101]
[0,13,17,18,19,15,16,14,12,101]
[0,20,24,25,27,29,30,28,26,23,22,21,101]
[0,98,96,95,94,92,93,97,100,99,101]
[0,101]
[0,101]
[0,101]
[0,101]
[0,32,33,31,35,37,38,39,36,34,101]
[0,81,78,76,71,70,73,77,79,80,101]
[0,101]
[0,43,42,41,40,44,46,45,48,51,50,52,49,47,101]
[0,90,87,86,83,82,84,85,88,89,91,101]
[0,101]
[0,101]
[0,101]
[0,101]
[0,101]
[0,67,65,63,62,74,72,61,64,68,66,69,101]
